In [1]:
!pip install earthengine-api
!earthengine authenticate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=QK6bZniGja34Ht9I4xjc17RmmzEWoas7bBFu9kbfa0E&tc=2UBtcoUpFqo4yOuqQPaYVerco8YDY3qGyz8h50zHQx0&cc=usC8E5IEMgMRIfikyoKhJlJLAzq2X4prkfFlUIO3Tmg

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AVHEtk7S7hWL6mGyg4S3Ss8MG1bJKl0_UVn67KPFvXbyrphpbDh5cazsrww

Successfully saved authorization token.


In [3]:
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.5/130.5 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.5 MB/s eta 0:00:00
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11135 sha256=b291d66d34b9f551b3b5c185d9b640f44436f4f3bad5cf5191256c

In [15]:
import ee
import geemap

# Initialize the Earth Engine API
ee.Initialize()

# Define the MinaisGerais region in Brazil (you can adjust coordinates to choose a specific section)
roi = ee.Geometry.Rectangle([-52.0, -22.0, -39.5, -13.5])

# Define the time periods for comparison
start_date1 = '2015-01-01'
end_date1 = '2015-12-31'
start_date2 = '2021-01-01'
end_date2 = '2021-12-31'

# Load Landsat 8 surface reflectance data for the two time periods
l8_sr1 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate(start_date1, end_date1).filterBounds(roi)
l8_sr2 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterDate(start_date2, end_date2).filterBounds(roi)

# Compute the median of each image collection
median1 = l8_sr1.median()
median2 = l8_sr2.median()

# Define a function to mask clouds and cloud shadows
def mask_clouds(image):
    cloud_shadow_bit_mask = 1 << 3
    clouds_bit_mask = 1 << 5
    qa = image.select('pixel_qa').toInt()
    mask = qa.bitwiseAnd(cloud_shadow_bit_mask).eq(0).And(qa.bitwiseAnd(clouds_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7'])


# Apply the cloud mask to each image collection
masked_median1 = mask_clouds(median1)
masked_median2 = mask_clouds(median2)

# Calculate Normalized Difference Vegetation Index (NDVI)
ndvi1 = masked_median1.normalizedDifference(['B5', 'B4'])
ndvi2 = masked_median2.normalizedDifference(['B5', 'B4'])

# Define an NDVI threshold to detect deforestation (can be adjusted for sensitivity)
ndvi_threshold = 0.3

# Detect areas with deforestation
deforestation = ndvi1.gt(ndvi_threshold).And(ndvi2.lt(ndvi_threshold))

# Create an interactive map
map_deforestation = geemap.Map()

# Add True Color composite images from both time periods
true_color_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3}
map_deforestation.addLayer(masked_median1.clip(roi), true_color_params, '2015 True Color')
map_deforestation.addLayer(masked_median2.clip(roi), true_color_params, '2021 True Color')

# Add deforestation layer to the map
map_deforestation.addLayer(deforestation.selfMask(), {'min': 0, 'max': 1, 'palette': ['red']}, 'Deforestation (Red)')

# Center the map on the region of interest and set the zoom level
map_deforestation.centerObject(roi, zoom=9)

# Add a layer control panel to the map
map_deforestation.addLayerControl()

# Display the map
map_deforestation


Map(center=[-17.81461151931359, -45.74999999999999], controls=(WidgetControl(options=['position', 'transparent…